<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/FaultRepair2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [2]:
# ─── SETUP FOR FAULT CONTROLLERS ───

import os, sys, subprocess

try:
    REPO_DIR = "/content/Cloud_Computing"
    LOGIC_DIR = os.path.join(REPO_DIR, "logic")

    # Step 1: Clone the repo if not present
    if not os.path.isdir(REPO_DIR):
        subprocess.run([
            "git", "clone", "-b", "develop",
            "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
            REPO_DIR
        ], check=True)

    # Step 2: Pull latest changes from develop
    subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

    # Step 3: Add logic folder to sys.path
    sys.path.append(LOGIC_DIR)

    # Step 4: Install dependencies
    %pip install -q importnb
    %pip install -q paho-mqtt
    %pip install -q -U gradio
    %pip install -q firebase
    %pip install requests beautifulsoup4
    %pip install -q matplotlib

    # Step 5: Import required notebooks
    from importnb import Notebook
    with Notebook():
        import Fault_controller as fc
        import repair_controller as rc
        import user_controller as uc

    print("✅ Fault setup completed successfully.")

except Exception as e:
    print("❌ Fault setup failed:", str(e))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.6 MB/s eta 0:00:00
omer_cohen 050-111-2222 200
shira_gold 050-222-3333 200
daniel_bar 050-333-4444 200
roni_benami 050-444-5555 200
lior_levi 050-555-6666 200
yael_elyashiv 050-666-7777 200
noa_katz 050-777-8888 200
matan_tal 050-888-9999 200
or_peled 050-999-0000 200
tom_segal 050-000-1111 200
Daniel Bar | 050-333-4444 | Robotics Engineer
Lior Levi | 050-555-6666 | Backend Developer
Matan Tal | 050-888-9999 | Mechanical Engineer
Noa Katz | 050-777-8888 | UX Designer
Omer Cohen | 050-111-2222 | Automation Engineer
Or Peled | 050-999-0000 | System Engineer
Roni Ben Ami | 050-444-5555 | QA Engineer
Shira Gold | 050-222-3333 | Electrical Engineer
Tom Segal | 050-000-1111 | Production Engineer
Yael Elyashiv | 050-666-7777 | Hardware Engineer
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 

In [19]:
import gradio as gr
from datetime import datetime

# ---------- Helpers ----------
def get_severity_color(severity):
    return {
        "Low": "#fff176",
        "Medium": "#ffb74d",
        "High": "#ef5350"
    }.get(severity, "#fff176")

def format_datetime(iso_str):
    dt = datetime.fromisoformat(iso_str)
    return dt.strftime("%-d/%-m, %-I:%M %p")

def split_sensor(sensor):
    parts = sensor.split()
    return " ".join(parts[:2]), " ".join(parts[2:]) if len(parts) > 2 else ""

def get_main_fault():
    """
    Returns the first inserted active fault from Firebase,
    and splits its title into sensor1 (first 2 words) and sensor2 (the rest).
    """
    faults = fc.fb.get_active_faults()
    if not isinstance(faults, dict) or not faults:
        return None, "", ""

    # Firebase dicts preserve insertion order (Python 3.7+)
    first_key = next(iter(faults))
    fault = faults[first_key]

    title_words = fault.get("title", "").split()
    sensor1 = " ".join(title_words[:2])
    sensor2 = " ".join(title_words[2:]) if len(title_words) > 2 else ""

    return fault, sensor1, sensor2

def active_faults_table_data(sort_mode="By Time"):
    """
    Returns a list of [Time, Title, Severity, Status] rows for active faults,
    sorted by the selected mode.
    """
    faults = fc.fb.get_active_faults()
    if not isinstance(faults, dict):
        return []

    severity_order = {"Low": 1, "Medium": 2, "High": 3}
    fault_items = list(faults.items())  # default: By Time (insertion order)

    if sort_mode == "Low → High":
        fault_items = sorted(fault_items, key=lambda x: severity_order.get(x[1].get("severity", ""), 0))
    elif sort_mode == "High → Low":
        fault_items = sorted(fault_items, key=lambda x: severity_order.get(x[1].get("severity", ""), 0), reverse=True)

    rows = []
    for _, fault in fault_items[1:]:  # Skip the main fault
        iso_ts = fault.get("timestamp", "")
        try:
            dt = datetime.fromisoformat(iso_ts)
            time_str = dt.strftime("%-d/%-m, %-I:%M%p").upper()
        except Exception:
            time_str = iso_ts[:16].replace("T", " ")

        rows.append([
            time_str,
            fault.get("title", ""),
            fault.get("severity", ""),
            fault.get("status", "")
        ])

    return rows

# ---------- Extracted ----------
presented_fault, sensor1, sensor2 = get_main_fault()
if presented_fault is None:
    raise gr.Error("No active faults found in Firebase.")

severity_color = get_severity_color(presented_fault["severity"])
title_words = presented_fault.get("title", "").split()
sensor1 = " ".join(title_words[:2])
sensor2 = " ".join(title_words[2:]) if len(title_words) > 2 else ""
time_str = format_datetime(presented_fault["timestamp"])
xp_value = {"Low": 50, "Medium": 100, "High": 200}.get(presented_fault["severity"], 50)

# ---------- Styles ----------
css = f"""
#main-fault {{
    background-color: {severity_color};
    color: white;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#main-fault * {{
    color: white !important;
}}
#main-fault ul {{
  padding-left: 15px !important;
  margin-top: 4px;
  margin-bottom: 0;
}}

#repair-box {{
    background-color: white;
    color: black;
    padding: 24px;
    border-radius: 20px;
    font-family: sans-serif;
}}
#repair-box * {{
    color: black !important;
}}
.repair-checkbox {{
  display: flex !important;
  flex-direction: row !important;
  gap: 20px;
  flex-wrap: wrap;
}}
"""

# ---------- App ----------
with gr.Blocks(css=css) as demo:

    # -------- First Row ----------
    with gr.Row():
        with gr.Column(elem_id="main-fault", scale=1):
            gr.Markdown("### FAULT DETECTED")
            gr.Markdown(f"**{time_str}**")
            gr.Markdown(f"### {sensor1}")
            gr.Markdown(sensor2)
            gr.Markdown("#### Suggested Actions:")
            for action in presented_fault["actions"]:
                gr.Markdown(f"- {action}")

        with gr.Column(elem_id="repair-box", scale=1):
            gr.Markdown("## REPAIR CHALLENGE")
            gr.Markdown(f"Earn {xp_value} XP by fixing")
            gr.Markdown(f"{sensor2 or presented_fault['sensor']} within 10 min")
            gr.Markdown("#### Repair Steps")

            repair_checklist = gr.CheckboxGroup(
                choices=presented_fault["actions"],
                value=[],
                interactive=False,
                label="",
                elem_classes=["repair-checklist"]
            )

            start_btn = gr.Button("Start Repair", visible=True)
            finish_btn = gr.Button("Finish Repair", visible=False, interactive=False)
            status_output = gr.Textbox(label="System Message", interactive=False)

    # -------- Second Row ----------
    with gr.Row():
        with gr.Column():
            with gr.Row():
                gr.Markdown("### Active Faults")
                sort_choice = gr.Dropdown(
                    choices=["By Time", "Low → High", "High → Low"],
                    value="By Time",
                    label="",
                    interactive=True,
                    scale=1,
                    elem_classes=["small-dropdown"]
                )

            table_headers = gr.Dataframe(
                headers=["Time", "Sensor", "Severity", "Status"],
                value=[],
                row_count=0,
                col_count=(4, "fixed"),
                interactive=False,
                label=""
        )


    # ---------- Logic ----------
    def start_real_repair():
        sensor = presented_fault["sensor"]
        msg = rc.start_repair(sensor)
        return gr.update(interactive=True), gr.update(visible=False), gr.update(visible=True), msg

    def handle_checkbox_update(selected):
        all_checked = set(selected) == set(presented_fault["actions"])
        return gr.update(interactive=all_checked)

    def complete_real_repair():
        sensor = presented_fault["sensor"]
        msg = rc.complete_repair(sensor)
        return gr.update(visible=False), gr.update(value="Start Repair", visible=True, interactive=True), msg

    def load_active_faults():
        rows = active_faults_table_data()
        return gr.update(value=rows)


    # ---------- Bindings ----------
    start_btn.click(
        fn=start_real_repair,
        outputs=[repair_checklist, start_btn, finish_btn, status_output]
    )

    repair_checklist.change(
        fn=handle_checkbox_update,
        inputs=repair_checklist,
        outputs=finish_btn
    )

    finish_btn.click(
        fn=complete_real_repair,
        outputs=[finish_btn, start_btn, status_output]
    )

    sort_choice.change(
    fn=active_faults_table_data,
    inputs=sort_choice,
    outputs=table_headers
    )

    demo.load(fn=lambda: active_faults_table_data("By Time"), outputs=table_headers)


demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b6f147c11f9d052f25.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
